In [1]:
import numpy as np 
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

### Defaul risk capital requirements under Standardised Approach

In [2]:
# Sample Portfolio 
# Our data with trade info and regulatory parameters assigned accordingly.
df1 = pd.read_excel(r'C:\Users\SAMIR SHARMA\Downloads\DRC_data.xlsx')
df1

,Issuer,Instrument,Type,Position,Exposure,Bucket,Rating,LGD,RW,Strike_Price,Notional_value,Mtm_value
0,Name 1,Stock 1,Stock,bought,LONG,Corporate,BBB,1.00,0.060,0.00,65,50.00
1,Name 1,Bought call Stock 1,Call_on_stock,bought,LONG,Corporate,BBB,1.00,0.060,70.00,0,1.56
2,Name 5,Sold call on stock 2,Call_on_stock,sold,SHORT,Corporate,A,1.00,0.030,90.08,0,-0.53
3,Name 6,Bought call on Stock 3,Call_on_stock,bought,LONG,Corporate,BB,1.00,0.150,80.00,0,3.64
4,Name 5,Bought put on stock 2,Put_on_stock,bought,SHORT,Corporate,A,1.00,0.030,85.48,0,6.08
5,Name 6,Sold put on stock 3,Put_on_stock,sold,LONG,Corporate,BB,1.00,0.150,110.00,0,-8.04
6,Name 1,Corporate covered bond,Bond,bought,LONG,Corporate,BBB,0.25,0.060,0.00,100,75.65
7,Name 2,Municipality bond,Bond,sold,SHORT,Municipality,AAA,0.75,0.005,0.00,-100,-89.47
8,Name 3,Sovereign bond,Bond,bought,LONG,Sovereign,AAA,0.75,0.005,0.00,100,82.22
9,Name 4,Corporate bond,Bond,sold,SHORT,Corporate,Unrated,0.75,0.150,0.00,-100,-83.82


In [3]:
## This code is for calculating NET JTDs, next we need to aggregate NETJTD according to Buckets.
class Instrument:
    def __init__(self, row):
        self.Position = row['Position']
        self.Mtm_value = row['Mtm_value']
        self.Value_at_default = 0

    def calculate_jtd(self):
        raise NotImplementedError("Subclasses must implement this method")


def calculate_net_jtd(df):
    # Create a new DataFrame to store the results
    result_df = pd.DataFrame(columns=['Issuer', 'NetJTD'])

    # Iterate over unique issuers
    for issuer in df['Issuer'].unique():
        # Select rows corresponding to the current issuer
        issuer_rows = df[df['Issuer'] == issuer]

        # Sum both 'JTD_long' and 'JTD_short' values for the current issuer
        net_jtd = issuer_rows['JTD_long'].sum() + issuer_rows['JTD_short'].sum()

        # Append the result to the new DataFrame
        result_df = result_df.append({'Issuer': issuer, 'NetJTD': net_jtd}, ignore_index=True)

    # Sort the DataFrame by issuer names
    result_df = result_df.sort_values(by='Issuer')

    return result_df
        
    

class Equity(Instrument):
    def calculate_jtd(self):
        if self.Position == 'sold' and row['Type'] == 'Stock':
            Market_value = self.Mtm_value * (-1)
            JTD_short = min(0, Market_value - self.Value_at_default)
            return JTD_short, 0
        else:
            JTD_long = max(0, self.Mtm_value - self.Value_at_default)
            return JTD_long, 0


class EquityOption(Instrument):
    def __init__(self, row):
        super().__init__(row)
        self.LGD = row['LGD']
        self.Type = row['Type']
        self.Strike_Price = row['Strike_Price']

    def calculate_jtd(self):
        if self.Position == 'sold' and self.Type == 'Call_on_stock':
            Value_at_default = max(0 - self.Strike_Price, 0)
            JTD_short = min(0, self.Mtm_value - Value_at_default)
            return 0, JTD_short
        elif self.Type == 'Put_on_stock' and self.Position == 'sold':
            Value_at_default = -(max(self.Strike_Price - 0, 0))
            JTD_long = max(0, self.Mtm_value - Value_at_default)
            return JTD_long, 0
        elif self.Type == 'Put_on_stock' and self.Position == 'bought':
            Value_at_default = max(self.Strike_Price, 0)
            JTD_short = min(0, self.Mtm_value - Value_at_default)
            return 0, JTD_short
        elif self.Position == 'bought' and self.Type == 'Call_on_stock':
            JTD_long = self.Mtm_value
            return JTD_long, 0


class OptionOnBond(Instrument):
    def __init__(self, row):
        super().__init__(row)
        self.LGD = row['LGD']
        self.Type = row['Type']
        self.Strike_Price = row['Strike_Price']
        self.Notional = row['Notional_value']

    def calculate_jtd(self):
        if self.Type == 'Bond' and self.Position == 'bought':
            Value_at_default = self.Notional * (1 - self.LGD)
            JTD_long = max(0, self.Mtm_value - Value_at_default)
            return JTD_long, 0
        elif self.Type == 'Bond' and self.Position == 'sold':
            Value_at_default = self.Notional * (1 - self.LGD)
            JTD_short = min(0, self.Mtm_value - Value_at_default)
            return 0, JTD_short
        elif self.Type == 'Put_on_bond' and self.Position == 'bought':
            Value_at_default = max(0, self.Strike_Price - abs(self.Notional) * (1 - self.LGD))
            JTD_short = min(0, self.Mtm_value - Value_at_default)
            return 0, JTD_short
        elif self.Type == 'Put_on_bond' and self.Position == 'sold':
            Value_at_default = -(self.Strike_Price - abs(self.Notional) * (1 - self.LGD))
            JTD_long = max(0, self.Mtm_value - Value_at_default)
            return JTD_long, 0


class CreditDefaultSwap(Instrument):
    def __init__(self, row):
        super().__init__(row)
        self.LGD = row['LGD']
        self.Strike_Price = row['Strike_Price']
        self.Notional = row['Notional_value']

    def calculate_jtd(self):
        if self.Position == 'bought':
            Value_at_default = abs(self.Notional) * self.LGD
            JTD_short = min(0, self.Mtm_value - Value_at_default)
            return 0, JTD_short
        elif self.Position == 'sold':
            Value_at_default = -self.Notional * self.LGD
            JTD_long = max(0, self.Mtm_value - Value_at_default)
            return JTD_long, 0


# Function to create the appropriate instrument object based on the row data
def create_instrument(row):
    Type = row['Type']
    if 'Stock' in Type:
        return Equity(row)
    elif 'Call_on_stock' in Type or 'Put_on_stock' in Type:
        return EquityOption(row)
    elif 'Bond' in Type or 'Put_on_bond' in Type:
        return OptionOnBond(row)
    elif 'CDS' in Type:
        return CreditDefaultSwap(row)
    else:
        raise ValueError(f"Unsupported instrument type: {Type}")


# Apply the function to calculate JTD for each row
jtd_results = [create_instrument(row).calculate_jtd() for _, row in df1.iterrows()]

# Extract JTD_long and JTD_short into separate lists
jtd_long, jtd_short = zip(*jtd_results)


df1['JTD_long'] = jtd_long
df1['JTD_short'] = jtd_short


equity_instance = Equity(df1.iloc[0])

# Calculate Net JTD using the calculate_net_jtd method
Net_JTD_df = calculate_net_jtd(df1)
Net_JTD_df

,Issuer,NetJTD
0,Name 1,52.21
3,Name 2,-126.39
4,Name 3,-2692.78
5,Name 4,51.62
1,Name 5,-79.93
2,Name 6,105.6
6,Name 7,3800.0


In [4]:
# Create a mapping dictionary from df1
issuer_bucket_mapping = dict(zip(df1['Issuer'], df1['Bucket']))

# Create a mapping dictionary for 'RW' in df1
rw_mapping = dict(zip(df1['Issuer'], df1['RW']))

"""
 A mapping dictionary (issuer_bucket_mapping) is created using the unique values in the 'Issuer' column of df1 as keys
 and the corresponding 'Bucket' values as values. 
 Then, the map function is used to create a new 'Bucket' column in Net_JTD_df by mapping the 'Issuer' values to
 their corresponding 'Bucket' values from the mapping dictionary.
 """ 

# Map the 'Bucket' values to Net_JTD_df based on 'Issuer' column
Net_JTD_df['Bucket'] = Net_JTD_df['Issuer'].map(issuer_bucket_mapping)

# Map the 'RW' values to Net_JTD_df based on 'Issuer' column
Net_JTD_df['RW'] = Net_JTD_df['Issuer'].map(rw_mapping)

# Reordering the columns
Net_JTD_df = Net_JTD_df[['Issuer','Bucket', 'RW', 'NetJTD']]
# Add a new column 'RW.NETJTD'
Net_JTD_df['RW * NETJTD'] = Net_JTD_df['RW'] * Net_JTD_df['NetJTD']
Net_JTD_df

,Issuer,Bucket,RW,NetJTD,RW * NETJTD
0,Name 1,Corporate,0.060,52.21,3.1326
3,Name 2,Municipality,0.005,-126.39,-0.63195
4,Name 3,Sovereign,0.005,-2692.78,-13.4639
5,Name 4,Corporate,0.150,51.62,7.743
1,Name 5,Corporate,0.030,-79.93,-2.3979
2,Name 6,Corporate,0.150,105.6,15.84
6,Name 7,Municipality,0.300,3800.0,1140.0


In [5]:
# Create new DataFrame based on existing columns
new_df = pd.DataFrame()

# Group by 'Bucket' and calculate sums
grouped = Net_JTD_df.groupby('Bucket')
new_df['RW.Net JTD(long)'] = grouped['RW * NETJTD'].apply(lambda x: x[x > 0].sum())
new_df['RW.Net JTD(short)'] = grouped['RW * NETJTD'].apply(lambda x: x[x < 0].sum())
new_df['Net JTD(long)'] = grouped['NetJTD'].apply(lambda x: x[x > 0].sum())
new_df['Net JTD(short)'] = grouped['NetJTD'].apply(lambda x: x[x < 0].sum())
new_df['HBR'] = new_df['Net JTD(long)'] / (new_df['Net JTD(long)'] + abs(new_df['Net JTD(short)']))
new_df['DRCb'] = new_df['RW.Net JTD(long)'] - abs(new_df['RW.Net JTD(short)']) * new_df['HBR']


# Reset index to have 'Bucket' as a regular column
new_df.reset_index(inplace=True)

# Set 'Bucket' as the index
new_df.set_index('Bucket', inplace=True)
print(f"Final Defaul Risk Capital Charge: ${new_df['DRCb'].sum()}")
new_df 

Final Defaul Risk Capital Charge: $1164.3684650305936


,RW.Net JTD(long),RW.Net JTD(short),Net JTD(long),Net JTD(short),HBR,DRCb
Bucket,,,,,,
Corporate,26.7156,-2.39790,209.43,-79.93,0.72377,24.980073
Municipality,1140.0000,-0.63195,3800.00,-126.39,0.96781,1139.388392
Sovereign,0.0000,-13.46390,0.00,-2692.78,0.00000,0.000000
